In [2]:
import pandas as pd

def hapus_kolom_tidak_digunakan(file_input, file_output, kolom_yang_digunakan):
    # Membaca file CSV dengan memisahkan kolom berdasarkan delimiter ';'
    df = pd.read_csv(file_input, delimiter=';')

    # Mengecek nama kolom yang ada dalam DataFrame
    print("Nama Kolom dalam DataFrame:", df.columns)

    try:
        # Menghapus kolom yang tidak digunakan
        df = df[kolom_yang_digunakan]

        # Menyimpan DataFrame yang sudah dimodifikasi ke file CSV baru
        df.to_csv(file_output, index=False)

        print(f"File {file_output} telah berhasil dibuat dengan kolom yang diinginkan.")
    except KeyError as e:
        print(f"Error: {e}. Pastikan nama kolom yang digunakan sesuai dengan nama kolom dalam file CSV.")

if __name__ == "__main__":
    # Ganti nama file_input.csv dengan nama file CSV yang ingin Anda proses
    file_input = 'dataset2.csv'

    # Ganti nama file_output.csv dengan nama file CSV yang ingin Anda hasilkan
    file_output = 'datasetfix2.csv'

    # Ganti kolom_yang_digunakan dengan daftar kolom yang ingin Anda pertahankan
    kolom_yang_digunakan = ['Sentiment', 'Text Tweet']

    # Memanggil fungsi untuk menghapus kolom yang tidak digunakan
    hapus_kolom_tidak_digunakan(file_input, file_output, kolom_yang_digunakan)


Nama Kolom dalam DataFrame: Index(['Id,Sentiment,Pasangan Calon,Text Tweet'], dtype='object')
Error: "None of [Index(['Sentiment', 'Text Tweet'], dtype='object')] are in the [columns]". Pastikan nama kolom yang digunakan sesuai dengan nama kolom dalam file CSV.


In [4]:
import numpy as np

TWEET_DATA = pd.read_csv("dataset2.csv", encoding = "ISO-8859-1")
TWEET_DATA['Text Tweet'].str.encode('ascii', 'ignore')
TWEET_DATA.head()

,ï»¿Id,Sentiment,Pasangan Calon,Text Tweet
0,1,negative,Agus-Sylvi,Banyak akun kloning seolah2 pendukung #agussil...
1,2,negative,Agus-Sylvi,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,3,negative,Agus-Sylvi,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,4,negative,Agus-Sylvi,Kasian oh kasian dengan peluru 1milyar untuk t...
4,5,negative,Agus-Sylvi,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...


In [6]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
TWEET_DATA['Text Tweet'] = TWEET_DATA['Text Tweet'].str.lower()


print('Case Folding Result : \n')
print(TWEET_DATA['Text Tweet'].head(5))
print('\n\n\n')

Case Folding Result : 

0    banyak akun kloning seolah2 pendukung #agussil...
1    #agussilvy bicara apa kasihan yaa...lap itu ai...
2    kalau aku sih gak nunggu hasil akhir qc tp lag...
3    kasian oh kasian dengan peluru 1milyar untuk t...
4    maaf ya pendukung #agussilvy..hayo dukung #ani...
Name: Text Tweet, dtype: object






In [7]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
TWEET_DATA['Text Tweet'] = TWEET_DATA['Text Tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

TWEET_DATA['Text Tweet'] = TWEET_DATA['Text Tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

TWEET_DATA['Text Tweet'] = TWEET_DATA['Text Tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

TWEET_DATA['Text Tweet'] = TWEET_DATA['Text Tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

TWEET_DATA['Text Tweet'] = TWEET_DATA['Text Tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

TWEET_DATA['Text Tweet'] = TWEET_DATA['Text Tweet'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

TWEET_DATA['tweet_tokens'] = TWEET_DATA['Text Tweet'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(TWEET_DATA['tweet_tokens'].head())
print('\n\n\n')

Tokenizing Result : 

0    [banyak, akun, kloning, seolah, pendukung, mul...
1    [bicara, apa, kasihan, yaalap, itu, air, matan...
2    [kalau, aku, sih, gak, nunggu, hasil, akhir, q...
3    [kasian, oh, kasian, dengan, peluru, milyar, u...
4       [maaf, ya, pendukung, hayo, dukung, diputaran]
Name: tweet_tokens, dtype: object






In [8]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

TWEET_DATA['tweet_tokens_fdist'] = TWEET_DATA['tweet_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(TWEET_DATA['tweet_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(banyak, 1), (akun, 1), (kloning, 1), (seolah...
1    [(bicara, 1), (apa, 1), (kasihan, 1), (yaalap,...
2    [(nunggu, 2), (kalau, 1), (aku, 1), (sih, 1), ...
3    [(kasian, 2), (oh, 1), (dengan, 1), (peluru, 1...
4    [(maaf, 1), (ya, 1), (pendukung, 1), (hayo, 1)...
Name: tweet_tokens_fdist, dtype: object


In [9]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')
print(len(list_stopwords))

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
len(list_stopwords)
# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopword.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
len(list_stopwords)
# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

TWEET_DATA['tweet_tokens_WSW'] = TWEET_DATA['tweet_tokens'].apply(stopwords_removal) 


print(TWEET_DATA['tweet_tokens_WSW'].head())

758
0    [akun, kloning, pendukung, menyerang, paslon, ...
1    [bicara, kasihan, yaalap, air, matanya, wkwkwkwk]
2    [nunggu, hasil, qc, nunggu, motif, cuitan, pas...
3    [kasian, kasian, peluru, milyar, rw, mempan, m...
4                 [maaf, pendukung, dukung, diputaran]
Name: tweet_tokens_WSW, dtype: object


In [10]:
import spacy
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Load model bahasa Indonesia dari spaCy
nlp_id = spacy.load("xx_ent_wiki_sm")

# Membuat objek Stemmer dari Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Normalisasi kata dengan spaCy Bahasa Indonesia
def normalized_term_id(document):
    # Tokenisasi dokumen menggunakan spaCy
    doc = nlp_id(document)
    
    # Normalisasi kata dalam dokumen
    normalized_document = [token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc]
    
    return normalized_document

# Normalisasi kata dengan Sastrawi
def stemmed_wrapper(term):
    return stemmer.stem(term)

# Membuat kamus untuk normalisasi kata
term_dict = {}

# Normalisasi kata dengan spaCy dan Sastrawi
for document in TWEET_DATA['tweet_tokens_WSW']:
    normalized_document_id = normalized_term_id(' '.join(document))
    normalized_document_stemmed = [stemmed_wrapper(term) for term in normalized_document_id]
    
    for term in normalized_document_stemmed:
        if term not in term_dict:
            term_dict[term] = ' '

# Menampilkan jumlah kata yang sudah dinormalisasi dan di-stem
print(len(term_dict))
print("------------------------")

# Melakukan stemming pada kamus term_dict
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term, ":", term_dict[term])

print(term_dict)
print("------------------------")

# Menerapkan term yang sudah di-stem pada dataframe
def get_stemmed_term(document):
    return [term_dict.get(term, term) for term in document]


TWEET_DATA['tweet_tokens_stemmed'] = TWEET_DATA['tweet_tokens_WSW'].apply(get_stemmed_term)
print(TWEET_DATA['tweet_tokens_stemmed'])


1
------------------------
 : 
{'': ''}
------------------------
0      [akun, kloning, pendukung, menyerang, paslon, ...
1      [bicara, kasihan, yaalap, air, matanya, wkwkwkwk]
2      [nunggu, hasil, qc, nunggu, motif, cuitan, pas...
3      [kasian, kasian, peluru, milyar, rw, mempan, m...
4                   [maaf, pendukung, dukung, diputaran]
                             ...                        
895                           [bpk, rspun, selfie, hand]
896    [merangkul, batas, usia, kelamin, hand, victor...
897    [jagoanku, dibidang, digital, smiling, face, w...
898                                                   []
899    [sandiaga, bangun, rumah, simpel, dibanding, t...
Name: tweet_tokens_stemmed, Length: 900, dtype: object


In [11]:
TWEET_DATA.to_csv("Text_Preprocessing2.csv")


In [14]:


# Membaca file CSV dan mengabaikan kolom indeks 'Unnamed: 0'
text_preprocessing_data = pd.read_csv('text_preprocessing2.csv', index_col=0)

# Memilih hanya kolom 'sentiment' dan 'tweet_tokens_stemmed'
selected_columns = text_preprocessing_data[['Sentiment', 'tweet_tokens_stemmed']]

# Menyimpan DataFrame yang telah dipilih ke dalam file CSV baru
selected_columns.to_csv('final_dataset2.csv', index=False)

# Menampilkan DataFrame yang telah dipilih
print(selected_columns)


    Sentiment                               tweet_tokens_stemmed
0    negative  ['akun', 'kloning', 'pendukung', 'menyerang', ...
1    negative  ['bicara', 'kasihan', 'yaalap', 'air', 'matany...
2    negative  ['nunggu', 'hasil', 'qc', 'nunggu', 'motif', '...
3    negative  ['kasian', 'kasian', 'peluru', 'milyar', 'rw',...
4    negative       ['maaf', 'pendukung', 'dukung', 'diputaran']
..        ...                                                ...
895  positive                 ['bpk', 'rspun', 'selfie', 'hand']
896  positive  ['merangkul', 'batas', 'usia', 'kelamin', 'han...
897  positive  ['jagoanku', 'dibidang', 'digital', 'smiling',...
898  positive                                                 []
899  positive  ['sandiaga', 'bangun', 'rumah', 'simpel', 'dib...

[900 rows x 2 columns]
